In [1]:
import pandas as pd
import sklearn.model_selection
import sklearn.metrics
import tqdm
import os
import numpy as np

In [3]:
result_folder = '/nas/home/cborrelli/speech_forensics/results'
param_dict = {'norm': ['minmax', 'zscore', 'nonorm'],
                  'filter_outliers': [True, False],
                  'rms_th': [0, 0.25, 0.50, 0.75],
                  'alg': ['svm', 'rf'],
                  'win_min_max': [True, False]
                  }
param_list = sklearn.model_selection.ParameterGrid(param_dict)

col = ['name', 'norm', 'filter_outlier', 'rms_th', 'alg', 'win_min_max', 'cl_balanced_accuracy', 'cl_f1',  
       'mcl_balanced_accuracy', 'mcl_f1', 'r_R2', 'r_mae']

results = pd.DataFrame(columns=col)

for params in tqdm.tqdm(param_list):
    res_file_name = 'res_norm-{}_outliers-{}_rms-{}_alg-{}_win-{}.pkl'.format(params['norm'],
                                                                              params['filter_outliers'],
                                                                              str(params['rms_th']).replace('.', ''),
                                                                              params['alg'],
                                                                              params['win_min_max'])
    res_df = pd.read_pickle(os.path.join(result_folder, res_file_name))
    
    
    y_cl_true = np.array(res_df['y_value'], dtype=np.float) >= 0.5
    cl_balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_cl_true, res_df['y_pred_cl'])
    cl_f1_score = sklearn.metrics.f1_score(y_cl_true, res_df['y_pred_cl'])
    
    y_mcl_true = np.array(res_df['y_label'], dtype=np.float) - 1  # labels for classification
    mcl_balanced_accuracy = sklearn.metrics.accuracy_score(y_mcl_true, res_df['y_pred_mcl'])
    mcl_f1= sklearn.metrics.f1_score(y_mcl_true, res_df['y_pred_mcl'], average='micro')
    
    y_r_true = np.array(res_df['y_value'], dtype=np.float)  #  values for regression
    r_R2 = sklearn.metrics.r2_score(y_r_true, res_df['y_pred_r'])
    r_mae = sklearn.metrics.mean_absolute_error(y_r_true, res_df['y_pred_r'])
    
    newrow = pd.DataFrame( [[res_file_name, params['norm'], params['filter_outliers'], params['rms_th'], params['alg'], 
                            params['win_min_max'], cl_balanced_accuracy, cl_f1_score, mcl_balanced_accuracy, mcl_f1,
                            r_R2, r_mae]],
                    columns=col)
    results = results.append(newrow, ignore_index=True)
    




    
    
    



 17%|█▋        | 16/96 [00:02<00:13,  5.84it/s]/nas/home/cborrelli/miniconda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
 18%|█▊        | 17/96 [00:02<00:12,  6.17it/s]/nas/home/cborrelli/miniconda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
 19%|█▉        | 18/96 [00:02<00:13,  5.91it/s]/nas/home/cborrelli/miniconda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
 20%|█▉        | 19/96 [00:03<00:12,  6.23it/s]/nas/home/cborrelli/miniconda/lib/python3.7/site-packages/sklearn/metrics/classificati

In [ ]:
results.sort_values(by='cl_balanced_accuracy', ascending=False)

In [ ]:
param_list.param_grid